In [46]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED
from models.shred_models import evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
train_size = 0.9
val_size = 0.05
test_size = 0.05

# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = train_size,
    val_size = val_size,
    test_size = test_size,
    compression = 200,
    scaling = True,
    time=np.arange(0, 1900),
    method="reconstruct"
)


# Add data to manager (no sensors)
input_folder = "data14fields_npz"

# Add data to manager (with sensors)
filename = 'ni_3D.npz'
file_path = os.path.join(input_folder, filename)
ni_3D = np.load(file_path)
ni_3D = ni_3D[ni_3D.files[0]]

ntimesteps = ni_3D.shape[0]
ni_3D_reconstruction_train_for_prediction = ni_3D[:-int(ntimesteps*test_size)]
print(ni_3D_reconstruction_train_for_prediction.shape)


manager.add_field(
    data=ni_3D_reconstruction_train_for_prediction,
    stationary_sensors=[(29, 201), (76, 26), (11, 247)],
    id = str(os.path.splitext(filename)[0])
)


(1900, 257, 256)


In [50]:
manager.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (user selected),"(29, 201)"
1,ni_3D,ni_3D-1,stationary (user selected),"(76, 26)"
2,ni_3D,ni_3D-2,stationary (user selected),"(11, 247)"


In [51]:
manager.sensor_measurements

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,5.263100e+16,5.625600e+16,5.009700e+16
1,1,5.315000e+16,5.802500e+16,5.040600e+16
2,2,5.405600e+16,5.980300e+16,4.995000e+16
3,3,5.479300e+16,5.928200e+16,4.755900e+16
4,4,5.432100e+16,5.721300e+16,4.576400e+16
...,...,...,...,...
1895,1895,7.886800e+16,1.061800e+17,6.299500e+16
1896,1896,7.676800e+16,1.102500e+17,5.421400e+16
1897,1897,7.432300e+16,1.165700e+17,4.766400e+16
1898,1898,7.760600e+16,1.178000e+17,4.605400e+16


In [52]:
# Get train/val/test datasets
train_set, val_set, test_set = manager.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor_dataset.X.shape)
print('train Y:', train_set.reconstructor_dataset.Y.shape)
print('val X:', val_set.reconstructor_dataset.X.shape)
print('val Y:', val_set.reconstructor_dataset.Y.shape)
print('test X:', test_set.reconstructor_dataset.X.shape)
print('test Y:', test_set.reconstructor_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([1710, 21, 3])
train Y: torch.Size([1710, 200])
val X: torch.Size([95, 21, 3])
val Y: torch.Size([95, 200])
test X: torch.Size([95, 21, 3])
test Y: torch.Size([95, 200])


In [53]:
num_epochs = 2000
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_set, val_set, num_epochs=num_epochs, patience=100, verbose=True)


Fitting Reconstructor...


Epoch 1/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 2/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 3/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 4/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

...

Epoch 1068/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 1069/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 1070/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Epoch 1071/2000:   0%|          | 0/27 [00:00<?, ?batch/s]

Early stopping triggered: patience threshold reached.


In [54]:
evaluate(shred = shred, test_dataset=test_set, data_manager=manager, uncompress=True, unscale = True)

,ni_3D
reconstruction,0.069192


In [55]:
# Initialize SHREDDataManager
manager2 = SHREDDataManager(
    lags = 20,
    train_size = train_size,
    val_size = val_size,
    test_size = test_size,
    compression = 200,
    scaling = True,
    time=np.arange(0, 2000),
    method="all"
)

manager2.add_field(
    data=ni_3D,
    stationary_sensors=[(29, 201), (76, 26), (11, 247)],
    id = str(os.path.splitext(filename)[0])
)


In [56]:
manager2.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (user selected),"(29, 201)"
1,ni_3D,ni_3D-1,stationary (user selected),"(76, 26)"
2,ni_3D,ni_3D-2,stationary (user selected),"(11, 247)"


In [57]:
manager2.sensor_measurements

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,5.263100e+16,5.625600e+16,5.009700e+16
1,1,5.315000e+16,5.802500e+16,5.040600e+16
2,2,5.405600e+16,5.980300e+16,4.995000e+16
3,3,5.479300e+16,5.928200e+16,4.755900e+16
4,4,5.432100e+16,5.721300e+16,4.576400e+16
...,...,...,...,...
1995,1995,7.625200e+16,1.036000e+17,4.160500e+16
1996,1996,7.924100e+16,1.044900e+17,4.053400e+16
1997,1997,8.920600e+16,1.165200e+17,4.258400e+16
1998,1998,8.696500e+16,1.371100e+17,4.459300e+16


In [58]:
# Get train/val/test datasets
train_set_2, val_set_2, test_set_2 = manager2.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set_2.reconstructor_dataset.X.shape)
print('train Y:', train_set_2.reconstructor_dataset.Y.shape)
print('val X:', val_set_2.reconstructor_dataset.X.shape)
print('val Y:', val_set_2.reconstructor_dataset.Y.shape)
print('test X:', test_set_2.reconstructor_dataset.X.shape)
print('test Y:', test_set_2.reconstructor_dataset.Y.shape)

print ('Predictor Data')
print('train X:', train_set_2.predictor_dataset.X.shape)
print('train Y:', train_set_2.predictor_dataset.Y.shape)
print('val X:', val_set_2.predictor_dataset.X.shape)
print('val Y:', val_set_2.predictor_dataset.Y.shape)
print('test X:', test_set_2.predictor_dataset.X.shape)
print('test Y:', test_set_2.predictor_dataset.Y.shape)

print ('Sensor Forecaster Data')
print('train X:', train_set_2.sensor_forecaster_dataset.X.shape)
print('train Y:', train_set_2.sensor_forecaster_dataset.Y.shape)
print('val X:', val_set_2.sensor_forecaster_dataset.X.shape)
print('val Y:', val_set_2.sensor_forecaster_dataset.Y.shape)
print('test X:', test_set_2.sensor_forecaster_dataset.X.shape)
print('test Y:', test_set_2.sensor_forecaster_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([1800, 21, 3])
train Y: torch.Size([1800, 200])
val X: torch.Size([100, 21, 3])
val Y: torch.Size([100, 200])
test X: torch.Size([100, 21, 3])
test Y: torch.Size([100, 200])
Predictor Data
train X: torch.Size([1800, 21, 3])
train Y: torch.Size([1800, 200])
val X: torch.Size([100, 21, 3])
val Y: torch.Size([100, 200])
test X: torch.Size([100, 21, 3])
test Y: torch.Size([100, 200])
Sensor Forecaster Data
train X: torch.Size([1800, 21, 3])
train Y: torch.Size([1800, 3])
val X: torch.Size([100, 21, 3])
val Y: torch.Size([100, 3])
test X: torch.Size([100, 21, 3])
test Y: torch.Size([100, 3])


In [59]:
# initialize SHRED
shred2 = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred2.fit(train_set_2, val_set_2, num_epochs=num_epochs, patience=100, verbose=True)


Fitting Reconstructor...


Epoch 1/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 2/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 3/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 4/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

...

Epoch 153/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 154/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 155/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Epoch 156/2000:   0%|          | 0/29 [00:00<?, ?batch/s]

Early stopping triggered: patience threshold reached.


In [60]:
evaluate(shred = shred2, test_dataset=test_set_2, data_manager=manager2, uncompress=True, unscale = True)

,ni_3D
reconstruction,0.070300
prediction,0.131140
forecast,0.132813
sensor_forecast,0.104509


In [61]:
test_set_2.reconstructor_dataset.X = test_set_2.predictor_dataset.X
test_set_2.reconstructor_dataset.Y = test_set_2.predictor_dataset.Y

In [62]:
# PREDICTION (NOT RECONSTRUCTION) WITH "SINGLE_MODEL" Version
evaluate(shred = shred, test_dataset=test_set_2, data_manager=manager, uncompress=True, unscale = True)

,ni_3D
reconstruction,0.144771
